In [1]:
import pandas as pd
from pandas import Series,DataFrame
import re
import datetime
from bs4 import BeautifulSoup
import requests
import pymongo

#寻找小区对应的区域和建造年份
#改进后的mongodb数据库里已经包含有quyu和district了
def get_data(x):
    try:
        html=requests.get('https://dl.lianjia.com/xiaoqu/rs'+x)
        soup=BeautifulSoup(html.text,'lxml')
        s=soup.select('.info .title a')
        for i in range(0,len(s)):
            if s[i].string==x:
                district=soup.select('.info .positionInfo .district')[i].string
                bizcircle=soup.select('.info .positionInfo .bizcircle')[i].string
                temp=soup.select('.info .positionInfo')[i].get_text()
                nianfen=re.search('\d+',temp).group()
                return x+' '+district+' '+bizcircle+' '+ str(nianfen)
        else:
            return None
    except IndexError as e:
        print(x)

pd.set_option('display.max_rows',None)
# #设置显示全部列，不省略
pd.set_option('display.max_columns',None)


#将pymongo数据转换成Dataframe
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["db_lianjia"]
mycol = mydb["chengjiao"]

x = list(mycol.find())
df=DataFrame(x)
#把没用的id删除掉
df=df.drop('_id',axis=1)    
df.drop_duplicates(inplace=True)
#把车位这一行删掉
df.drop(index=df[df['area']=='车位'].index,inplace=True) 
# df.dropna(how='all',inplace=True)

In [2]:
df['area']=df['area'].apply(lambda x: float(re.search("\d+(\.\d+)?",x).group()))
tofloat=lambda x:float(x)
df['unitPrice']=df['unitPrice'].apply(tofloat)
df['totalPrice']=df['totalPrice'].apply(tofloat)
df['dealDate']=pd.to_datetime(df['dealDate'])
# df['dealCycle']=df['dealCycle'].map(lambda x:int(re.search('\d+',x).group()))
df['guaPai']=df['guaPai'].apply(lambda x: float(re.search("\d+(\.\d+)?",x).group()))

In [3]:
#找出挂牌价和成交价差距大的房子
f=df[((df['guaPai']-df['totalPrice'])/df['guaPai']>0.05) & (df['dealDate'].map(lambda x:x.year)==2022)].sort_values(by=['title','dealDate'])
f['chazhi']=round((f['guaPai']-f['totalPrice'])/f['guaPai'],2)

In [ ]:
f[(f['dealDate'].map(lambda x:x.year)==2022) & (f['dealDate'].map(lambda x:x.month)==5)].sort_values(by='chazhi',ascending=False)

In [ ]:
#找出各个销售的销售量，目前看工人村单个销售卖的还真多
#功能保存，链家上的销售未必就是卖房的销售
s=df.groupby(['seller'])[['seller']].count()
s.index.name='num'
s=s.sort_values(by=['seller'],ascending=False)

In [ ]:
#找出各小区的销售数量，先忽略时间
# xiaoqu=df.groupby(['title'])[['title']].count()
xiaoqu=df['title'].groupby([df['title'],df['dealDate'].apply(lambda x:x.year)]).count()
xiaoqu.index.names=['title','year']
result=list(set(list(map(lambda x:re.search('悦泰(\w+)?',x),xiaoqu.index.levels[0]))))
result.remove(None)
r=[i.group() for i in result]

In [ ]:
s1=xiaoqu.loc[r[1]]
s2=xiaoqu.loc[r[0]]
s1.name=r[1]
s2.name=r[0]
print(s1,s2)


In [ ]:
#找出某个小区的销售情况
print(get_data('第五郡五号地'))
df_wu=df[df['title'].apply(lambda x: '第五郡' in x and '公寓' not in x)]
df_wu=df_wu.sort_values(by='dealDate')
df_wu=df_wu.reset_index(drop=True)
func=lambda x:str((x.year)*100+(x.month))[0:4]+'-'+str((x.year)*100+(x.month))[4:]
df_wu['month']=df_wu['dealDate'].map(func)
df_wu['title'].groupby(df_wu['month']).count()
# df[df['title']=='金海花园西园'][['title']].count()

In [ ]:
#找出小区每年均价，按月标本太少，不太准确
df_wu['unitPrice'].groupby(df_wu['dealDate'].map(lambda x:x.year)).describe()

In [ ]:
#找出小区的成交周期
df_wu['dealCycle'].groupby(df_wu['dealDate'].apply(lambda x:x.year)).describe()

In [ ]:
#找出房子里只有一天成交周期的房子
df[(df['dealCycle'].map(lambda x:x==1)) & (df['dealDate'].apply(lambda x:x.year)==2021)].head()

In [ ]:
#备用功能        
# list=[]
# for i in df['title'].drop_duplicates():
#     g=get_data(i)
#     if g is not None:
#         list.append(g)

# %run main.py
#df.to_json(path_or_buf='data/test.json',orient='index',force_ascii=False)
#检查area数据
# for i in range(0,len(df)):
#     result=df.iloc[i]
#     if re.search(r'\d+(\.\d+)?(平米)',result[2]) is None:
#         print(df.iloc[i])
#         df.drop([i])
# #检查挂牌数据
# for i in range(0,len(df)):
#     result=df.iloc[i]
#     if re.search('挂牌*',result[7]) is None:
#         print(str(i))
        # df.drop([i])

In [ ]:
#根据已有数据算全部小区
dd=df['unitPrice'].groupby(df['dealDate'].apply(lambda x:x.year)).describe()
dd[dd.index>2015]

In [4]:
#2021年每月成交数
count=df['quyu'].groupby([df['quyu'].apply(lambda x:x)!='',df['dealDate'].apply(lambda x:x.year)==2021,df['dealDate'].apply(lambda x:x.month)]).count()
s=count[True][True]
s

dealDate
1    1371
2    1544
3    4052
4    3178
5    2627
6     623
7    2504
8    1834
9      37
Name: quyu, dtype: int64

In [ ]:
#2021年每月均价
count=df['unitPrice'].groupby([df['dealDate'].apply(lambda x:x.year)==2021,df['dealDate'].apply(lambda x:x.month)]).mean()
count[True]

In [ ]:
#2021年每月成交面积
count=df['area'].groupby([df['dealDate'].apply(lambda x:x.year)==2021,df['dealDate'].apply(lambda x:x.month)]).sum()
count[True]

In [3]:
count=df['title'].groupby([df['dealDate'].apply(lambda x:x.year)==2021,df['dealDate'].apply(lambda x:x.month)==9,df['dealDate'].apply(lambda x:x.day)]).count()
count[True][True]

dealDate
1    37
Name: title, dtype: int64